In [2]:
!pip install langchain
!pip install langchain-community
!pip install datasets
# !pip install -U datasets
!pip install unstructured
!pip install faiss-cpu
!pip install peft

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlspace 0.23.4 requires pandas==1.5.0, but you have pandas 2.1.1 which is incompatible.
client-lib 0.3.9 requires typing-extensions==4.10.0, but you have typing-extensions 4.12.2 which is incompatible.
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.15.2-py3-none-any.whl (411 kB)


In [2]:
!unzip docs.zip

Archive:  docs.zip
  inflating: docs/overview.md        
  inflating: docs/api_quickstart.md  
  inflating: docs/limits.md          


In [ ]:
import os
import json
from typing import Optional

import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling,
    pipeline
)
from datasets import Dataset, load_dataset, concatenate_datasets
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

from peft import LoraConfig, get_peft_model, PeftModel


BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
TOKENIZER = BASE_MODEL


SBERQUAD_NAME = "kuznetsoffandrey/sberquad"


SEED = 42
SAMPLE_SIZE_PER_DS = None


domain_docs_path = "./docs/"
domain_qa_file = "./domain_faq_ru.json"

sft_output_dir = "./sft_checkpoints_ru_lora/"
retrieval_index_path = "./vector_index_ru/"


if not os.path.exists(domain_qa_file):
    example_faq = [
        {"question": "Как получить API-ключ?", "answer": "API-ключ можно получить в личном кабинете в разделе 'Настройки' -> 'API'"},
        {"question": "Где найти документацию по методам?", "answer": "Документация находится в каталоге ./docs/ или на сайте в разделе 'Документация'"}
    ]
    with open(domain_qa_file, 'w', encoding='utf-8') as f:
        json.dump(example_faq, f, ensure_ascii=False, indent=2)
    print(f"Создан примерный файл FAQ: {domain_qa_file}")



tokenizer = AutoTokenizer.from_pretrained(TOKENIZER, trust_remote_code=True)

def format_prompt_for_sft(question: str, answer: str, context: str = "") -> str:
    messages = [
        {"role": "system", "content": (
            "Ты - полезный AI-ассистент. Отвечай на вопросы, используя предоставленный контекст. "
            "Если информации нет в контексте, скажи об этом и предложи свой вариант ответа."
        )},
        {"role": "user", "content": f"Контекст:\n{context}\n\nВопрос: {question}"},
        {"role": "assistant", "content": answer}
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )



def load_sberquad(split: str = "train") -> Dataset:
    ds = load_dataset(SBERQUAD_NAME, split=split)
    if SAMPLE_SIZE_PER_DS and SAMPLE_SIZE_PER_DS < len(ds):
        ds = ds.shuffle(seed=SEED).select(range(SAMPLE_SIZE_PER_DS))

    def to_text(rec):
        question = rec["question"]
        context = rec["context"]
        answer = rec["answers"]["text"][0] if rec["answers"]["text"] else ""
        return {"text": format_prompt_for_sft(question, answer, context)}

    return ds.map(to_text, remove_columns=ds.column_names)


def load_domain_faq(path: str) -> Dataset:
    with open(path, 'r', encoding='utf-8') as f:
        items = json.load(f)
    data = [{"text": format_prompt_for_sft(it['question'], it['answer'])} for it in items]
    return Dataset.from_list(data)


def tokenize_dataset(ds: Dataset) -> Dataset:
    def tok(ex):
        return tokenizer(ex["text"], truncation=True, padding=True, max_length=512, return_tensors="pt")
    return ds.map(tok, batched=True, remove_columns=["text"])


def run_lora_sft(tokenized_ds: Dataset, model, tokenizer):
    lora_config = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj","v_proj"], lora_dropout=0.05, bias="none")
    peft_model = get_peft_model(model, lora_config)
    peft_model.print_trainable_parameters()

    args = TrainingArguments(
        save_strategy="epoch",
        output_dir=sft_output_dir,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=16,
        num_train_epochs=2,
        logging_steps=100,
        save_steps=500,
        fp16=True,
        eval_strategy="no",
        remove_unused_columns=False,
        save_safetensors=False
    )
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
    trainer = Trainer(model=peft_model, args=args, train_dataset=tokenized_ds, data_collator=data_collator)
    trainer.train()
    peft_model.save_pretrained(sft_output_dir)



def build_retrieval_index(docs_path: str, idx_path: str):
    loader = DirectoryLoader(docs_path, glob='**/*')
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
    store = FAISS.from_documents(texts, embeddings)
    os.makedirs(idx_path, exist_ok=True)
    store.save_local(idx_path)
    return store



def load_bot(model_path: str, index_path: str):
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        trust_remote_code=True,
        device_map="cpu"
    )
    model = PeftModel.from_pretrained(
        base_model,
        model_path,
        trust_remote_code=True,
        device_map="cpu"
    )
    model = model.merge_and_unload()
    
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        truncation=True,
        do_sample=False,
        temperature=0.0,
        top_p=None,
        top_k=None,
    )
    llm = HuggingFacePipeline(pipeline=pipe)

    vectorstore = FAISS.load_local(
        index_path,
        HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru"),
        allow_dangerous_deserialization=True
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    def format_prompt(context, question):
        messages = [
            {
                "role": "system", 
                "content": "Ты - полезный AI-ассистент. Отвечай на вопросы, используя предоставленный контекст. Если информации нет в контексте, обязательно скажи об этом и предложи свой вариант ответа."
            },
            {
                "role": "user", 
                "content": f"Контекст:\n{context}\n\nВопрос: {question}"
            }
        ]
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

    prompt = PromptTemplate(
                input_variables=["context", "question"],
                template=format_prompt("{context}", "{question}")
            )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False,
        chain_type_kwargs={
            "prompt": prompt
        }
    )
    
    return qa_chain

In [ ]:
sber_train = load_sberquad("train")
sber_val   = load_sberquad("validation")
sber_test  = load_sberquad("test")
sber = concatenate_datasets([sber_train, sber_val, sber_test]).shuffle(seed=SEED)

faq  = load_domain_faq(domain_qa_file)

ds     = concatenate_datasets([sber, faq])
tok_ds = tokenize_dataset(ds)

In [ ]:
# Обучаем LoRA
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, trust_remote_code=True)
run_lora_sft(tok_ds, model, tokenizer)

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.714000
200,1.496300
300,1.478300
400,1.473900
500,1.475900
600,1.473700
700,1.468100
800,1.467300
900,1.460000
1000,1.461300


In [ ]:
build_retrieval_index(domain_docs_path, retrieval_index_path)

bot = load_bot(sft_output_dir, retrieval_index_path)
example_questions = [
    "Какое расстояние от Земли до Луны?",
    "Как подключиться к API нашего продукта?",
    "Какие ограничения по длине записи в системе?",
]
for q in example_questions:
    output = bot.run(q)
    print(f"\nВопрос: {q}")
    print("Ответ:", output)

Device set to use cpu
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn


Вопрос: Какое расстояние от Земли до Луны?
Ответ: <|im_start|>system
Ты - полезный AI-ассистент. Отвечай на вопросы, используя предоставленный контекст. Если информации нет в контексте, обязательно скажи об этом и предложи свой вариант ответа.<|im_end|>
<|im_start|>user
Контекст:
Обзор продукта

ProductX — это облачная платформа, позволяющая хранить и обрабатывать данные с задержкой менее 50 мс.

Основные возможности: 1. REST‑API и клиентские SDK (Python, JS, Go). 2. Гибкая система ролей и доступов. 3. Поддержка потоковой записи и триггеров Webhook. 4. SLA 99.95 % при базе Standard.

Архитектура

Платформа построена на микросервисах, каждый из которых масштабируется автоматически в Kubernetes‑кластере. Данные хранятся в распределённом хранилище на базе ClickHouse.

Быстрый старт с API

1. Получите API‑ключ

Перейдите в личный кабинет.

Откройте раздел Настройки → API.

Нажмите Сгенерировать ключ. Сохраните его — повторно он не отображается.

2. Первый запрос

bash curl -H "Authorizati

/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



Вопрос: Как подключиться к API нашего продукта?
Ответ: <|im_start|>system
Ты - полезный AI-ассистент. Отвечай на вопросы, используя предоставленный контекст. Если информации нет в контексте, обязательно скажи об этом и предложи свой вариант ответа.<|im_end|>
<|im_start|>user
Контекст:
Быстрый старт с API

1. Получите API‑ключ

Перейдите в личный кабинет.

Откройте раздел Настройки → API.

Нажмите Сгенерировать ключ. Сохраните его — повторно он не отображается.

2. Первый запрос

bash curl -H "Authorization: Bearer <API_KEY>" \ -H "Content-Type: application/json" \ -d '{"record": {"id": "42", "payload": "Hello!"}}' \ https://api.productx.io/v1/records Ожидаемый ответ 201 Created: json { "status": "success", "record_id": "42" }

3. Проверка статуса

bash curl -H "Authorization: Bearer <API_KEY>" https://api.productx.io/v1/records/42/status

Обзор продукта

ProductX — это облачная платформа, позволяющая хранить и обрабатывать данные с задержкой менее 50 мс.

Основные возможности: 1. REST

/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jovyan/.mlspace/envs/zhogov_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



Вопрос: Какие ограничения по длине записи в системе?
Ответ: <|im_start|>system
Ты - полезный AI-ассистент. Отвечай на вопросы, используя предоставленный контекст. Если информации нет в контексте, обязательно скажи об этом и предложи свой вариант ответа.<|im_end|>
<|im_start|>user
Контекст:
Ограничения и квоты

Параметр Значение по умолчанию Макс. размер записи 256 KB Макс. запросов в минуту 600 Макс. одновременных потоков 10

Замечание: квоты можно увеличить через службу поддержки.

Ограничения содержимого

Запрещены бинарные данные без Base64.

Максимальная глубина вложенных структур JSON — 10 уровней.

Значения полей id должны быть уникальны в пределах пространства вашего аккаунта.

Обзор продукта

ProductX — это облачная платформа, позволяющая хранить и обрабатывать данные с задержкой менее 50 мс.

Основные возможности: 1. REST‑API и клиентские SDK (Python, JS, Go). 2. Гибкая система ролей и доступов. 3. Поддержка потоковой записи и триггеров Webhook. 4. SLA 99.95 % при базе Standa